# API : Prédiction du nombre de vélos loués à SEOUL sur une période 

Dans la partie "Projet_prediction_des_velos_seouls.ipynb" nous avons pu étudier les données et créer un modèle adapté à la problématique de récupération du nombre de vélos loués à seoul sur une période donnée. Ce modèle a été enregistré et nous allons le réutiliser sous forme d'API afin de réaliser des prédictions.

Une API fonctionne généralement via un système de Client-Serveur. Le serveur va contenir notre modèle et va prédire le nombre de vélos loués à partir des données fournis par le client lors d'une requete.

Les étapes de fonctionnement sont :
- On lance le serveur sur une adresse IP et un port, on met en place un point d'acces au serveur via /api
- Le client va emettre une requete sur notre serveur via son url "http://IP:Port/api" (la requete contient les données nécessaire pour prédire)
- Le serveur reçoit la requete avec les données (GET) , ils utilisent le modèle pour prédire un résultat, le serveur retourne la prédiction (POST)
- Le client reçoit la réponse de la prédiction


Pour mettre en place notre APi, plusieurs solutions s'offraient à nous, parmis celles-ci Flask est celle que nous avons retenu.

# Serveur Flask

Le serveur Flask fonctionne généralement sous un .py qu'on vient lancer depuis la console via les commandes suivantes :
- pip install flask
- set FLASK_APP=serveur.py
- flask run

Le fichier d'extension python (.py) va contenir le code de notre serveur.

Cependant afin de s'affranchir de la console, nous avons proposé de lancer le serveur directement depuis jupyter notebook à l'aide d'un Thread.

In [1]:
#Cree un serveur local
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle
import json
import pandas as pd

#Le faire fonctionner en thread
import socket
import threading
import uuid
from typing import Any, Callable, cast, Optional

from flask import Flask, abort, jsonify
from flask_cors import cross_origin
from werkzeug.serving import run_simple

app = Flask('DataServer')

#Port de notre serveur
_port: int = 6666

#notre modèle
filename = 'catboost_model'
loaded_model = pickle.load(open(filename, 'rb'))

#dans notre serveur
#Si un client fait une requete sur /api 
#On prend les données envoyés par la requete 
#On retourne le résultat de la prédiction
@app.route('/api/', methods=['GET','POST'])
def makecalc():
    data = request.get_json()
    y_test_predict = loaded_model.predict(data)
    return jsonify(str(int(y_test_predict[0])))

#Initiatisation du serveur
def _init_port() -> int:
    """Creates a random free port."""
    # see https://stackoverflow.com/a/5089963/2297345
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('localhost', _port))

    port = sock.getsockname()[1]
    sock.close()
    return cast(int, port)

#Lancement du serveur sous forme de thread afin de fonctionner dans jupyter notebook
def _start_sever() -> None:
    """Starts a flask server in the background."""
    _port = _init_port()
    thread = threading.Thread(target=lambda: run_simple('localhost', _port, app))
    thread.start()

#Lance le serveur
_start_sever()

 * Running on http://localhost:6666/ (Press CTRL+C to quit)


Notre serveur est maintenant actif sur http://localhost:6666/, pour faire une prédiction il suffit de faire une requête sur l'adresse http://localhost:6666/api/

# Client

Le client est la personne souhaitant à partir de ses données réaliser une prédiction. Pour réaliser celle-ci, le client va faire une requete sur notre serveur lancé au préalable et récupérer le résultat pour l'afficher.

In [4]:
import requests
import json

#Adresse url de notre API
url = 'http://localhost:6666/api/'

#Nos données d'entrées du modèle servant à prédire le nombre de vélo loué :
#Hour
#Temperature(°C)
#Humidity(%)
#Wind speed (m/s)
#Visibility (10m)
#Dew point temperature(°C)
#Solar Radiation (MJ/m2)
#Rainfall(mm)
#Snowfall (cm)
#Seasons
#Holiday
#Functioning Day
#Year
#Month
#Day
#DayOfWeek
#isWeekEnd
data = [[0, -5.2, 37, 2.2, 2000, -17.6, 0.00, 0.0, 0.0, 0, False, True, 2017, 12, 1, 4, False]]

#Transforme notre list en json
j_data = json.dumps(data)
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

#Envoie la requete contenant les données à notre API
#Recupere le résultat de l'API
r = requests.post(url, data=j_data, headers=headers)
print("Le nombre de vélo loué pour cette heure et ce jour là sera de :" + r.text)

127.0.0.1 - - [07/Jan/2021 17:50:40] "POST /api/ HTTP/1.1" 200 -


Le nombre de vélo loué pour cette heure et ce jour là sera de :"234"

